In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Conv2D, Reshape
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.backend import clear_session

import matplotlib.pyplot as plt
import numpy as np

In [2]:
!wget repo.gorchilov.net/datasets/genetics.tar.gz -O /tmp/genetics.tar.gz
!tar -xf /tmp/genetics.tar.gz -C /tmp

import h5py

train = h5py.File('/tmp/train.h5', mode='r')
train_X = train['data'][:]
train_y = train['labels'][:]

test = h5py.File('/tmp/test.h5', mode='r')
test_X = test['data'][:]
test_y = test['labels'][:]

--2020-12-05 13:50:11--  http://repo.gorchilov.net/datasets/genetics.tar.gz
Resolving repo.gorchilov.net (repo.gorchilov.net)... 185.97.75.52
Connecting to repo.gorchilov.net (repo.gorchilov.net)|185.97.75.52|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63942414 (61M) [application/octet-stream]
Saving to: ‘/tmp/genetics.tar.gz’

/tmp/genetics.tar.g 100%[===================>]  60.98M  11.1MB/s    in 5.6s    

2020-12-05 13:50:17 (10.9 MB/s) - ‘/tmp/genetics.tar.gz’ saved [63942414/63942414]



non_one_hot = np.array([argmax(x, axis=0).numpy() for x in train_X])

In [3]:
expanded = np.expand_dims(train_X, axis=-1)

In [4]:
expanded.shape

(100000, 4, 1000, 1)

In [51]:
model = Sequential([
                    Conv2D(4, kernel_size=(4, 4), activation='relu', input_shape=(4, 1000, 1)),
                    Reshape((997, 4)),
                    Bidirectional(LSTM(64)),
                    Dense(512, activation='relu'),
                    Dense(256, activation='relu'),
                    Dense(128, activation='relu'),
                    Dense(1, activation='relu')
])

model.summary()

In [52]:
model.compile(optimizer=RMSprop(), loss='mse')

In [ ]:
clear_session()

history = model.fit(expanded[:10000],
                    train_y[:10000],
                    epochs=2000
)

Epoch 1/2000
63/63 [==============================] - 3s 55ms/step - loss: 14547271.0000
Epoch 2/2000
63/63 [==============================] - 3s 54ms/step - loss: 14188624.0000
Epoch 3/2000
63/63 [==============================] - 3s 54ms/step - loss: 14173685.0000
Epoch 4/2000
63/63 [==============================] - 3s 54ms/step - loss: 14182396.0000
Epoch 5/2000
63/63 [==============================] - 3s 53ms/step - loss: 14182665.0000
Epoch 6/2000
63/63 [==============================] - 3s 54ms/step - loss: 14152539.0000
Epoch 7/2000
63/63 [==============================] - 3s 52ms/step - loss: 14174750.0000
Epoch 8/2000
63/63 [==============================] - 3s 52ms/step - loss: 14160950.0000
Epoch 9/2000
63/63 [==============================] - 3s 52ms/step - loss: 14159036.0000
Epoch 10/2000
63/63 [==============================] - 3s 52ms/step - loss: 14165882.0000
Epoch 11/2000
63/63 [==============================] - 3s 53ms/step - loss: 14144375.0000
Epoch 12/2000
63/63

In [ ]:
start = 0
end = 3000

plt.title('all epochs')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(range(start, end), history.history['loss'][start:end])
plt.show()

In [ ]:
model.evaluate(np.expand_dims(test_X, axis=-1)[:10000], test_y[:10000])

In [ ]:
model.evaluate(expanded[:10000], train_y[:10000])

In [ ]:
model.save('model')

!tar -zcvf model.tar.gz model

# from google.colab import files
# files.download('model.tar.gz') 